#### Import Packages

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

#### Use BeautifulSoup to Get Data from Wiki

In [33]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki = requests.get(url).text
soup = BeautifulSoup (wiki, 'lxml')

#### Webscraping

In [54]:
col = ['Postalcode','Borough','Neighborhood']
toronto_data = pd.DataFrame(columns = col)

table_contents = []
table = soup.find('table')

for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
        
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'}

df.head()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,15,103
top,M5S,North York,Upper Rouge
freq,1,24,1


#### Groupby to Merge Multiple Neighborhoods in the Same Borough

In [58]:
df_grouped = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,15,103
top,M5S,North York,Upper Rouge
freq,1,24,1


##### Upon inspectin the data, there is no borough "Not assigned"

In [56]:
df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [57]:
df_grouped.shape

(103, 3)